# 뉴럴 네트워크 학습 알고리즘 구현

## Import modules

In [1]:
import tensorflow as tf

In [2]:
import tensorflow as tf
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices()) # print avaliable CPU, GPU list
print("───────────────────────────────────────")
print("am I using GPU?: ", bool(tf.config.list_physical_devices('GPU'))) # check GPU

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4055166204808021784
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5725224960
locality {
  bus_id: 1
  links {
  }
}
incarnation: 553317765384741827
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3070 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]
───────────────────────────────────────
am I using GPU?:  True


## 하이퍼파라미터 정의

In [3]:
EPOCHS = 10

## 네트워크 구조 정의

In [4]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super().__init__() # tf.keras.Model 의 초기자
        # 벡터 형태의 Flatten
        self.flatten = tf.keras.layers.Flatten(input_shape=(28,28))
        self.dense1 = tf.keras.layers.Dense(32, activation="relu")
        self.dense2 = tf.keras.layers.Dense(64, activation="relu")
        self.dense3 = tf.keras.layers.Dense(128, activation="relu")
        self.dense4 = tf.keras.layers.Dense(256, activation="relu")
        self.dense5 = tf.keras.layers.Dense(10, activation="softmax")
        
    def call(self, x, traing=None, mask=None):
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        return self.dense5(x)

## 학습 함수 구현

In [5]:
@tf.function # decorator가 쓰여진 함수는 최적화하여 돌아감
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
    # Feed Forward
    with tf.GradientTape() as tape:     # 해당 context에서 진행되는 모든 연산은 Gradient 계산
        predictions = model(images)     # BatchSize X 10
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    # Back Propagation
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, predictions)

## 테스트 함수 구현

In [6]:
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images)     # BatchSize X 10
    loss = loss_object(labels, predictions)
    
    test_loss(loss)
    test_accuracy(labels, predictions)

## 데이터 불러오기

In [7]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(1024).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

## 모델 생성

In [8]:
model = MyModel()

## 손실함수 및 최적화 알고리즘 정의

In [9]:
print(y_train)
print(y_train.shape)

[5 0 4 ... 5 6 8]
(60000,)


In [10]:
# labels과 predictions의 입력을 받음
# Sparse 표현의 Categorical data
# One-Hot은 CategoricalCrossentropy()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

## 성능 지표 정의

In [11]:
train_loss = tf.keras.metrics.Mean(name="train_loss")
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="train_accuracy")

test_loss = tf.keras.metrics.Mean(name="test_loss")
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="test_accuracy")

## 학습 루프 구현

In [12]:
for epoch in range(EPOCHS):
    # 1 epoch 구현
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)
    for images, labels in test_ds:
        test_step(model, images, labels, loss_object, test_loss, test_accuracy)
        
    template = "Epoch: {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}"
    print(template.format(epoch+1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))
    train_loss.reset_states()
    train_accuracy.reset_states()

    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch: 1, Loss: 0.2757263779640198, Accuracy: 91.56500244140625, Test Loss: 0.15087364614009857, Test Accuracy: 95.38999938964844
Epoch: 2, Loss: 0.1348193883895874, Accuracy: 95.85333251953125, Test Loss: 0.11993402987718582, Test Accuracy: 96.5
Epoch: 3, Loss: 0.10877823084592819, Accuracy: 96.67166137695312, Test Loss: 0.14518208801746368, Test Accuracy: 95.88999938964844
Epoch: 4, Loss: 0.08960619568824768, Accuracy: 97.19499969482422, Test Loss: 0.12995578348636627, Test Accuracy: 96.08000183105469
Epoch: 5, Loss: 0.07672464847564697, Accuracy: 97.5816650390625, Test Loss: 0.12437032163143158, Test Accuracy: 96.41999816894531
Epoch: 6, Loss: 0.06778595596551895, Accuracy: 97.81500244140625, Test Loss: 0.11323846876621246, Test Accuracy: 96.5
Epoch: 7, Loss: 0.059386078268289566, Accuracy: 98.02999877929688, Test Loss: 0.1218288317322731, Test Accuracy: 96.73999786376953
Epoch: 8, Loss: 0.055264923721551895, Accuracy: 98.22166442871094, Test Loss: 0.1061599925160408, Test Accuracy: